In [1]:
# libraries
import cv2
import numpy as np
import pickle
from WrnchPose import Pose2d
import math

Using TensorFlow backend.


In [2]:
def load_RF(filename='pose_model.pkl'):
    with open(filename, 'rb') as file:
        classifier = pickle.load(file)
        
    return classifier

In [3]:
# initialize wrnchAI model
wrnchPose = Pose2d(license_key='319074-C0E4EA-494F8E-E93B65-F8A713-AC63B0')

Initializing networks...
Initialization done!


In [4]:
# joints indexes in wrnchAI
jt_def = wrnchPose.joint_definition
joints_list = ['NOSE', 'NECK', 'RSHOULDER', 'RELBOW', 'RWRIST',\
               'LSHOULDER', 'LELBOW', 'LWRIST', 'REYE', 'LEYE']
joints_wrnch = list(map(jt_def.get_joint_index, joints_list))

In [5]:
def calAngle(x1, y1, x2, y2):
    ''' return angle in degrees '''
    if y2 < 0 or y1 < 0 or x2 < 0 or x1 < 0:
        return 90.
    
    theta = math.atan2(y2 - y1, x2 - x1) # radian
    theta = (theta * 180)/np.pi
    return theta

In [12]:
# 'Nose_Neck_A','Rshoulder_Neck_A','Lshoulder_Neck_A','Rshoulder_Relbow_A','Lshoulder_Lelbow_A'
# list of indexes for joints between which angle is calculated
joints_for_angle = [(0, 1), (2, 1), (5, 1), (2, 3), (5, 6)]
joints_for_angle += [(3, 4), (6, 7)]
# joints_for_angle = []

In [13]:
def get_joints_angles(wrnchPose, frame, joints):
    coor = list()
    
    wrnchPose.estimator.process_frame(frame, wrnchPose.options) # run model
    human2d = wrnchPose.estimator.humans_2d() # detected coordinates
    
    for human in human2d:
        points = human.joints()
        w,h = frame.shape[1], frame.shape[0]
        
        for jidx in joints:
            coor.append(points[jidx *2]) #x
            coor.append(points[jidx *2 + 1]) #y
        
#         coor.append(w)
#         coor.append(h)
#         coor.append(posture)
        break # for one person only
    
    return coor

In [14]:
classifier = load_RF()

In [16]:
# starting video testing
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # get frmae
    ret, frame = cap.read()
    # get coordinates 
    coor = get_joints_angles(wrnchPose, frame, joints_wrnch)
    # no prediction when no person
    if len(coor) > 10: # no person no points so need enough
        # calculate and append angles
        for i, j in joints_for_angle:
            coor.append( calAngle( coor[2*i], coor[2*i+1], coor[2*j], coor[2*j+1]))
        # get prediction
        coor = np.asarray(coor) # convert to numpy array
        coor = np.expand_dims(coor, axis=0) # expand dimension to fit classifier
        pose_res = classifier.predict(coor) # pred pose
        # horizontal flip 
        frame = cv2.flip(frame, 1)
        # adjust classes because of flip (interchange left and right poses)
        pose_res = pose_res[0]
        if 'left' in pose_res:
            pose_res = pose_res.replace('left', 'right')
        elif 'right' in pose_res:
            pose_res = pose_res.replace('right', 'left')
        # write result on image
        frame = cv2.putText(frame, "Pose: " + pose_res,\
                            (int(frame.shape[1]*0.15), int(frame.shape[0]*0.15)),\
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    # display the image
    cv2.imshow("FitSit", frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
            
cv2.destroyAllWindows()
cap.release()

In [12]:
cap.release()